<a href="https://colab.research.google.com/github/cpython-projects/python_da_06_11_25/blob/main/lesson_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Легенда

Ви — аналітик даних у роздрібній компанії, що розвиває мобільний застосунок. Команда продукту хоче зрозуміти:

* як користувачі взаємодіють із застосунком: скільки установок, скільки переглядають товари, скільки купують;
* де втрачаються користувачі (на якому етапі воронки);
* яка ефективність різних каналів залучення.

## Таблиці бази даних

**1. `app_sessions`** — установки застосунку

| Поле                  | Тип SQL        | Опис                                            |
| --------------------- | -------------- | ----------------------------------------------- |
| `session_id`          | `VARCHAR(20)`  | Унікальний ідентифікатор установки              |
| `device_code`         | `VARCHAR(20)`  | Ідентифікатор пристрою                          |
| `first_seen`          | `DATE`         | Дата установки                                  |
| `os_type`             | `VARCHAR(10)`  | Платформа (`iOS`, `Android`)                    |
| `acquisition_channel` | `VARCHAR(50)`  | Канал установки (`Organic`, `Facebook` і т.д.)  |
| `cpi_uah`             | `NUMERIC(6,2)` | Вартість установки (Cost Per Install) у гривнях |

> `cpi_uah` (Cost Per Install) — це сума, яку компанія платить рекламній платформі (наприклад, Facebook, Google Ads) за те, що користувач встановив застосунок по рекламі.
> * Використовується для оцінки ефективності каналів і розрахунку окупності (ROI)

---

**2. `product_views_log`** — перегляди товарів

| Поле          | Тип SQL       | Опис                           |
| ------------- | ------------- | ------------------------------ |
| `device_code` | `VARCHAR(20)` | Пристрій                       |
| `view_date`   | `DATE`        | Дата перегляду                 |
| `platform`    | `VARCHAR(10)` | Платформа                      |
| `view_count`  | `INTEGER`     | Кількість переглянутих товарів |

---

**3. `devices_users_map`** — відповідність `device_code` і `user_uuid`

| Поле          | Тип SQL       | Опис                                 |
| ------------- | ------------- | ------------------------------------ |
| `device_code` | `VARCHAR(20)` | Пристрій                             |
| `user_uuid`   | `VARCHAR(20)` | Користувач (присвоюється при логіні) |

> Користувач може не авторизуватися — тоді `user_uuid` відсутній.

---

**4. `orders_log`** — покупки

| Поле         | Тип SQL         | Опис                                     |
| ------------ | --------------- | ---------------------------------------- |
| `user_uuid`  | `VARCHAR(20)`   | Унікальний ID авторизованого користувача |
| `order_time` | `DATE`          | Дата замовлення                          |
| `total_uah`  | `NUMERIC(10,2)` | Сума покупки у гривнях                   |


## Зв’язки таблиць

| Зв’язок                                                    | Опис                             |
| ---------------------------------------------------------- | -------------------------------- |
| `app_sessions.device_code = devices_users_map.device_code` | Зв’язок установки з користувачем |
| `devices_users_map.user_uuid = orders_log.user_uuid`       | Хто зробив замовлення            |
| `product_views_log.device_code = app_sessions.device_code` | Хто переглядав товари            |

## Шлях користувача

```
ВСТАНОВИВ → ПЕРЕГЛЯДАВ → АВТОРИЗУВАВСЯ → КУПИВ
(app_sessions) → (product_views_log) → (devices_users_map) → (orders_log)
```

## SQL-запити для створення таблиць

```sql
DROP TABLE IF EXISTS orders_log;
DROP TABLE IF EXISTS devices_users_map;
DROP TABLE IF EXISTS product_views_log;
DROP TABLE IF EXISTS app_sessions;

CREATE TABLE app_sessions (
    session_id VARCHAR(10) PRIMARY KEY,
    device_code VARCHAR(20) UNIQUE,
    first_seen DATE,
    os_type VARCHAR(10),
    acquisition_channel VARCHAR(50),
    cpi_uah NUMERIC(6, 2)
);

CREATE TABLE product_views_log (
    device_code VARCHAR(20),
    view_date DATE,
    platform VARCHAR(10),
    view_count INTEGER,
    FOREIGN KEY (device_code) REFERENCES app_sessions(device_code)
);

CREATE TABLE devices_users_map (
    device_code VARCHAR(20),
    user_uuid VARCHAR(20) UNIQUE,
    FOREIGN KEY (device_code) REFERENCES app_sessions(device_code)
);

CREATE TABLE orders_log (
    user_uuid VARCHAR(20),
    order_time DATE,
    total_uah NUMERIC(10, 2),
    FOREIGN KEY (user_uuid) REFERENCES devices_users_map(user_uuid)
);
```

In [ ]:
## Підключення до бази даних

In [1]:
DB_USER = "prog_academy_da_8936_user"
DB_PASS = "ih9DVSH5hmpm2DvvES0wIRFSnOrkJ2FZ"
DB_HOST = "dpg-d6g4hu0gjchc73d78ju0-a.oregon-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "prog_academy_da_8936"

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [4]:
from sqlalchemy import inspect
inspector = inspect(engine)
print(inspector.get_table_names())

['app_sessions', 'product_views_log', 'devices_users_map', 'orders_log']


## SELECT, FROM, WHERE — основа SQL-запиту

Кожен SQL-запит починається з чіткої структури:

```sql
SELECT [що вибрати]
FROM [звідки взяти]
WHERE [які рядки відфільтрувати]
```

### SELECT

* Вказує, **які стовпці** ми хочемо отримати
* Можна вказати `*`, щоб взяти всі стовпці (на практиці — лише для відладки)
* Підтримує **вирази** (арифметика, функції, перейменування через `AS`)

> ❗ **Порядок рядків не гарантований!**
> Якщо важливий порядок — використовуйте `ORDER BY`

### FROM

* Обов’язково вказує, **з якої таблиці** брати дані
* Може бути не одна таблиця (пізніше буде `JOIN`)

### WHERE

* Фільтрація рядків до всіх інших операцій (до `GROUP BY`, `HAVING`)
* Працює лише з рядками, які **існують** у таблиці

> ❗ **WHERE не може використовувати агрегатні функції** (наприклад, `AVG()`)

In [ ]:
query = """
"""

## Приклад небезпечного коду (SQL-ін’єкція)

Цей запит буде:

```sql
SELECT * FROM app_sessions
WHERE first_seen >= '2024-03-01' OR '1'='1'
```

`OR '1'='1'` завжди істинне, отже фільтр `first_seen >= ...` не працює — повертаються всі рядки, що небезпечно.

## Безпечний параметризований запит

## Логічні оператори AND, OR, IN, BETWEEN, LIKE

* `AND`, `OR` — дозволяють об’єднувати умови
* `AND` — обидві частини мають бути істинними
* `OR` — достатньо, щоб одна була істинною

> **Використовуйте дужки!** Логіка без дужок може бути несподіваною

### Приклад використання AND


### Оператор IN

Спрощує множинні порівняння (альтернатива багатьом OR)

### Використання параметрів із SQLAlchemy для IN

### BETWEEN a AND b

Діапазон значень включно. Зручно для дат і чисел

### LIKE — пошук за шаблоном

* `%` — будь-яка кількість будь-яких символів
* `_` — один будь-який символ

---

### Особливості LIKE у PostgreSQL

* `LIKE` — **чутливий до регістру**
* Для нечутливого пошуку використовуйте `ILIKE`

---

### Приклади LIKE:

| Умова          | Знайде рядки, де...              |
| -------------- | -------------------------------- |
| `LIKE 'Alex%'` | починається з `Alex`             |
| `LIKE '%son'`  | закінчується на `son`            |
| `LIKE '%lex%'` | містить `lex` у будь-якому місці |
| `LIKE '%'`     | всі рядки                        |

---

### Приклад регістронезалежного пошуку:

```sql
SELECT * FROM products
WHERE name ILIKE '%phone%';
```

---

### Символ `_` у LIKE

| Умова         | Знайде                              |
| ------------- | ----------------------------------- |
| `LIKE '_ex'`  | `Lex`, `Rex`, але не `Alex`         |
| `LIKE 'A__x'` | `Alex`, `Abbx`, але не `Ax`, `Alxx` |

## ORDER BY, LIMIT

* `ORDER BY` — сортування результату
* За замовчуванням — за зростанням (`ASC`)
* Можна вказати `DESC` — за спаданням
* `LIMIT` — обмежує кількість рядків (часто з `ORDER BY` для топ-N)

## IS NULL — робота з пропущеними значеннями

* `NULL` — "нічого" або "невідомо"
* Порівнювати `= NULL` не можна, треба `IS NULL` або `IS NOT NULL`

## DISTINCT — видалення дублікатів

* Видаляє повні дублікати рядків
* Працює на всю комбінацію стовпців, а не на окремі

## Задачі

**Задача 1.** Знайти всі установки з лютого 2024 року по каналах `'Organic'` або `'Referral'`.


**Задача 2.** Вивести топ-5 найдорожчих установок за `cpi_uah`.


**Задача 3.** Знайти унікальні платформи, на які встановлювали застосунок.


**Задача 4.** Показати всі `device_code`, які зустрічаються у переглядах з `view_count > 30`.


**Задача 5.** Вивести замовлення між 1 лютого і 1 березня 2024 року.


**Задача 6.** Знайти пристрої без прив’язки до `user_uuid`.